In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import torch.optim as optim
import torch.nn as nn

from qibo import Circuit, gates, hamiltonians, set_backend, construct_backend

from qiboml.models.encoding import PhaseEncoding
from qiboml.models.decoding import Expectation
from qiboml.interfaces.pytorch import QuantumModel
from qiboml.operations.differentiation import PSR, Jax

2025-05-15 11:49:13.741837: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747302553.761699   84462 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747302553.767614   84462 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747302553.782456   84462 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747302553.782487   84462 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747302553.782491   84462 computation_placer.cc:177] computation placer alr

### Construct model

In [2]:
set_backend("qiboml", platform="pytorch")

nqubits = 2
nlayers = 3

# Encoding layer
encoding_circ = PhaseEncoding(
    nqubits=nqubits, 
    encoding_gate=gates.RX
)

# Trainable layer
def trainable_circuit(entanglement=True):
    trainable_circ  = Circuit(nqubits)
    for q in range(nqubits):
        trainable_circ.add(gates.RY(q=q, theta=np.random.randn()))
        trainable_circ.add(gates.RZ(q=q, theta=np.random.randn()))
    if nqubits > 1 and entanglement:
        [trainable_circ.add(gates.CNOT(q%nqubits, (q+1)%nqubits) for q in range(nqubits))]
    return trainable_circ

[Qibo 0.2.19|INFO|2025-05-15 11:49:16]: Using qiboml (pytorch) backend on cpu


### Test function

In [7]:
def compute_gradient_wrt_x(
    execution_backend,
    differentiation_rule,
    trainable_circ_list,
):
    """Mount the model, execute it and compute gradient w.r.t. input data."""
    # Decoding layer
    decoding_circ = Expectation(
        nqubits=nqubits, 
        backend=execution_backend
    )

    structure = []
    for i, circ in enumerate(trainable_circ_list):
        structure.extend([
            encoding_circ, circ
        ])

    # The whole model
    model = QuantumModel(
        circuit_structure=structure,
        decoding=decoding_circ,
        differentiation=differentiation_rule,
    )

    data = torch.tensor([0.3, 0.9], requires_grad=True)
    
    output = model(data)
    print(output)
    output.backward()
    
    print(f"Grad via {differentiation_rule} on {execution_backend} backend:", data.grad)
    print(model.circuit_parameters.grad)

In [8]:
# Construct a list of trainable layers so that the params are the same in all tests
trainable_circ_list = []
for _ in range(nlayers):
    trainable_circ_list.append(trainable_circuit(entanglement=True))

### Execute on different backends and with different diffrules

In [9]:
compute_gradient_wrt_x(
    execution_backend=construct_backend("numpy"),
    differentiation_rule=PSR(),
    trainable_circ_list=trainable_circ_list,
)
compute_gradient_wrt_x(
    execution_backend=construct_backend("qiboml", platform="pytorch"),
    differentiation_rule=None,
    trainable_circ_list=trainable_circ_list,
)
compute_gradient_wrt_x(
    execution_backend=construct_backend("numpy"),
    differentiation_rule=Jax(),
    trainable_circ_list=trainable_circ_list,
)

tensor([[-0.4918]], grad_fn=<QuantumModelAutoGradBackward>)
Grad via PSR() on numpy backend: tensor([-1.2792,  1.5955])
tensor([-1.3993e-01, -6.9612e-01, -9.6972e-01, -1.2516e+00,  1.1733e+00,
        -1.0823e-01,  5.0452e-01, -7.4237e-02,  4.2487e-03,  1.1401e-08,
         7.5779e-01, -5.7285e-10])
tensor([[-0.4918]], dtype=torch.float64, grad_fn=<ViewBackward0>)
Grad via None on qiboml (pytorch) backend: tensor([-1.2792,  1.5955])
tensor([-1.3993e-01, -6.9612e-01, -9.6972e-01, -1.2516e+00,  1.1733e+00,
        -1.0823e-01,  5.0452e-01, -7.4237e-02,  4.2487e-03,  3.0692e-10,
         7.5779e-01, -1.6375e-08])
tensor([[-0.4918]], grad_fn=<QuantumModelAutoGradBackward>)
Grad via Jax() on numpy backend: tensor([-1.2792,  1.5955])
tensor([-1.3993e-01, -6.9612e-01, -9.6972e-01, -1.2516e+00,  1.1733e+00,
        -1.0823e-01,  5.0452e-01, -7.4237e-02,  4.2487e-03, -4.9625e-18,
         7.5779e-01, -2.3158e-18])


In [6]:
data = torch.tensor([0.3, 0.9], requires_grad=True)
data.requires_grad, data.is_leaf

(True, True)